In [13]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import json
import logging
from tqdm import tqdm
from datetime import datetime

# Configuration des chemins
base_dir = Path().resolve().parent.parent
sys.path.append(str(base_dir / "src"))

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)






The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

# ÉTAPE 1: CONFIGURATION DES SOURCES RSS


RSS_SOURCES = [
    # 🇫🇷 Presse française généraliste
    "https://feeds.lemonde.fr/lemonde/actualites-en-continu/",
    "https://www.franceinter.fr/rss/a-la-une.xml",
    "https://feeds.feedburner.com/lepoint/actualites",
    "https://rss.lefigaro.fr/lefigaro/laune",
    "https://www.nouvelobs.com/rss.xml",
    "https://www.lesechos.fr/rss/rss_une.xml",
    "https://www.huffingtonpost.fr/feeds/index.xml",
    "https://www.lci.fr/rss/rss.xml",
    "https://www.francetvinfo.fr/titres.rss",

    #  Presse internationale (anglophone + francophone)
    "https://rss.cnn.com/rss/edition.rss",
    "https://feeds.bbci.co.uk/news/world/rss.xml",
    "https://www.reutersagency.com/feed/?best-topics=top-news&post_type=best",
    "https://www.npr.org/rss/rss.php?id=1001",
    "https://www.rfi.fr/fr/rss",  # Radio France Internationale
    "https://www.france24.com/fr/rss",
    "https://www.courrierinternational.com/feed/all/rss.xml",
    "https://www.swissinfo.ch/service/fra/rssxml/rss-sni-newsreel.xml",
    "https://rss.lematin.ch/rss/lematin/suisse",

    #  Science & Tech
    "https://feeds.feedburner.com/TechCrunch/",
    "https://rss.lesechos.fr/rss/lesechos/actualite/sciences-tech.xml",
    "https://www.futura-sciences.com/rss/actualites.xml",
    "https://www.sciencesetavenir.fr/rss.xml",
    "https://www.01net.com/rss/actualites/",
    "https://www.zdnet.fr/feeds/rss/actualites/",

    
     "https://towardsdatascience.com/feed",  
     "https://www.artificialintelligence-news.com/feed/"
]

print(f" {len(RSS_SOURCES)} sources RSS configurées")





 26 sources RSS configurées


In [15]:
# ÉTAPE 2: IMPORTS ET INITIALISATION  
print("\n ÉTAPE 2: Initialisation des composants")

from config.database import engine, SessionLocal
from data.models import Base
from data.database_manager import DatabaseManager
from data.rss_collector import RSSCollector
from data.article_parser import ArticleParser

# Création des tables
Base.metadata.create_all(bind=engine)
print(" Tables de base de données créées")

# Initialisation des composants
session = SessionLocal()
db = DatabaseManager(session=session)
# Pour collecter les 7 derniers jours
collector = RSSCollector(sources=RSS_SOURCES, days_back=7)

parser = ArticleParser(language='fr')





 ÉTAPE 2: Initialisation des composants

INFO:data.rss_collector: Collecte articles depuis: 2025-07-19 08:20



 Tables de base de données créées


In [16]:
# ÉTAPE 3: COLLECTE RSS
print(f"\n ÉTAPE 3: Collecte depuis {len(RSS_SOURCES)} flux RSS")

articles_rss = collector.fetch_feeds()
print(f" {len(articles_rss)} articles récupérés depuis les flux RSS")

if len(articles_rss) == 0:
    print(" ERREUR: Aucun article récupéré !")
    print(" Vérifiez votre connexion internet et les URLs RSS")
else:
    # Aperçu des articles collectés
    print(f"\n Aperçu des 3 premiers articles:")
    for i, article in enumerate(articles_rss[:3]):
        print(f"  {i+1}. {article['title'][:60]}...")
        print(f"     Source: {article['source']}")
        print(f"     URL: {article['url']}")





 ÉTAPE 3: Collecte depuis 26 flux RSS

INFO:data.rss_collector: Lecture flux RSS: https://feeds.lemonde.fr/lemonde/actualites-en-continu/


INFO:data.rss_collector: 0 articles récupérés de https://feeds.lemonde.fr/lemonde/actualites-en-continu/
INFO:data.rss_collector: Lecture flux RSS: https://www.franceinter.fr/rss/a-la-une.xml
INFO:data.rss_collector: 0 articles récupérés de https://www.franceinter.fr/rss/a-la-une.xml
INFO:data.rss_collector: Lecture flux RSS: https://feeds.feedburner.com/lepoint/actualites
INFO:data.rss_collector: 0 articles récupérés de https://feeds.feedburner.com/lepoint/actualites
INFO:data.rss_collector: Lecture flux RSS: https://rss.lefigaro.fr/lefigaro/laune
INFO:data.rss_collector: 0 articles récupérés de https://rss.lefigaro.fr/lefigaro/laune
INFO:data.rss_collector: Lecture flux RSS: https://www.nouvelobs.com/rss.xml
INFO:data.rss_collector: 190 articles récupérés de https://www.nouvelobs.com/rss.xml
INFO:data.rss_collector: Lecture flux RSS: https://www.lesechos.fr/rss/rss_une.xml
INFO:data.rss_collector: 0 articles récupérés de https://www.lesechos.fr/rss/rss_une.xml
INFO:data.rss_collector

 547 articles récupérés depuis les flux RSS

 Aperçu des 3 premiers articles:
  1. 7 très bons livres de poche à lire cet été, sur la plage ou ...
     Source: https://www.nouvelobs.com/rss.xml
     URL: https://www.nouvelobs.com/culture/20250726.OBS106223/7-tres-bons-livres-de-poche-a-lire-cet-ete-sur-la-plage-ou-ailleurs.html
  2. Léonie Pernet, Africa Express, Tami Neilson, Billie Marten… ...
     Source: https://www.nouvelobs.com/rss.xml
     URL: https://www.nouvelobs.com/culture/20250726.OBS106222/leonie-pernet-africa-express-tami-neilson-billie-marten-les-disques-a-ecouter-ou-pas-ce-week-end.html
  3. L’horoscope d’Abigail Assor du 27 juillet au 2 août...
     Source: https://www.nouvelobs.com/rss.xml
     URL: https://www.nouvelobs.com/horoscope/20250726.OBS106221/l-horoscope-d-abigail-assor-du-27-juillet-au-2-aout.html


In [17]:
# ÉTAPE 4: ENRICHISSEMENT AVEC CONTENU COMPLET
print(f"\n ÉTAPE 4: Récupération du contenu complet")

enriched_articles = []
success_count = 0
fail_count = 0

for article in tqdm(articles_rss, desc="Parsing articles"):
    try:
        # Récupération du texte complet
        full_text = parser.extract_text(article['url'])
        
        if full_text:
            article['text'] = full_text
            success_count += 1
        else:
            article['text'] = article.get('summary', '')  # Fallback
            fail_count += 1
            
        enriched_articles.append(article)
        
    except Exception as e:
        logger.warning(f"Erreur parsing {article['url']}: {e}")
        article['text'] = article.get('summary', '')
        enriched_articles.append(article)
        fail_count += 1

print(f" Texte complet récupéré : {success_count}")
print(f"  Échecs de parsing : {fail_count}")





 ÉTAPE 4: Récupération du contenu complet


Parsing articles:   2%|▏         | 11/547 [00:04<03:25,  2.61it/s]INFO:data.article_parser:Téléchargement de l'article : https://www.nouvelobs.com/teleobs/20250725.OBS106212/la-prisonniere-de-bordeaux-hafsia-herzi-isabelle-huppert-une-amitie-improbable.html
ERROR:data.article_parser:Échec parsing https://www.nouvelobs.com/teleobs/20250725.OBS106212/la-prisonniere-de-bordeaux-hafsia-herzi-isabelle-huppert-une-amitie-improbable.html : Article `download()` failed with 406 Client Error: Not Acceptable for url: https://www.nouvelobs.com/teleobs/20250725.OBS106212/la-prisonniere-de-bordeaux-hafsia-herzi-isabelle-huppert-une-amitie-improbable.html on URL https://www.nouvelobs.com/teleobs/20250725.OBS106212/la-prisonniere-de-bordeaux-hafsia-herzi-isabelle-huppert-une-amitie-improbable.html
Parsing articles:   5%|▍         | 26/547 [00:09<03:15,  2.66it/s]INFO:data.article_parser:Téléchargement de l'article : https://www.nouvelobs.com/justice/20250725.OBS106196/bachar-al-assad-face-a-la-justice

 Texte complet récupéré : 432
  Échecs de parsing : 115


In [18]:
# ÉTAPE 5: SAUVEGARDE EN BASE DE DONNÉES
print(f"\n ÉTAPE 5: Sauvegarde en base de données")

saved_count = 0
duplicate_count = 0

for article in tqdm(enriched_articles, desc="Sauvegarde"):
    try:
        article_id = db.create_article(article)
        if article_id:
            saved_count += 1
        else:
            duplicate_count += 1
    except Exception as e:
        logger.error(f"Erreur sauvegarde: {e}")

print(f" Articles sauvés : {saved_count}")
print(f"  Doublons ignorés : {duplicate_count}")


 ÉTAPE 5: Sauvegarde en base de données


Sauvegarde: 100%|██████████| 547/547 [00:00<00:00, 721.74it/s] 

 Articles sauvés : 547
  Doublons ignorés : 0


In [19]:
# ÉTAPE 6: VÉRIFICATION FINALE
print(f"\n ÉTAPE 6: Vérification finale")

all_articles = db.get_all_articles()
print(f" Total articles en base : {len(all_articles)}")

if len(all_articles) > 0:
    print(f" Exemples d'articles en base :")
    for i, article in enumerate(all_articles[:3]):
        print(f"  {i+1}. ID: {article['id']} - {article['title'][:50]}...")
        print(f"     Source: {article['source']}")
        print(f"     Texte: {len(article.get('text', ''))} caractères")



 ÉTAPE 6: Vérification finale
 Total articles en base : 586
 Exemples d'articles en base :
  1. ID: 1 - Almost a third of people in Gaza not eating for da...
     Source: https://feeds.bbci.co.uk/news/world/rss.xml
     Texte: 4358 caractères
  2. ID: 2 - 'I witnessed war crimes' in Gaza, former worker at...
     Source: https://feeds.bbci.co.uk/news/world/rss.xml
     Texte: 488 caractères
  3. ID: 3 - Thailand warns clashes with Cambodia could 'move t...
     Source: https://feeds.bbci.co.uk/news/world/rss.xml
     Texte: 5108 caractères


In [20]:
# ÉTAPE 7: EXPORT DE SAUVEGARDE
print(f"\n ÉTAPE 7: Export de sauvegarde")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = base_dir / "data" / "exports"
output_dir.mkdir(parents=True, exist_ok=True)


 ÉTAPE 7: Export de sauvegarde


In [21]:
# Export JSON des articles bruts
raw_articles_file = output_dir / f"raw_articles.json"
with open(raw_articles_file, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=2)

print(f" Articles exportés : {raw_articles_file}")


 Articles exportés : C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\exports\raw_articles.json


In [22]:
# ANALYSE DU FILTRAGE TEMPOREL


# Résumé de la période
date_info = collector.get_date_range_summary()
print(f" Période de collecte:")
print(f"   Début: {date_info['start_date'][:16]}")
print(f"   Fin:   {date_info['end_date'][:16]}")
print(f"   Durée: {date_info['days_back']} jours ({date_info['total_hours']}h)")


 Période de collecte:
   Début: 2025-07-19T08:20
   Fin:   2025-07-26T08:23
   Durée: 7 jours (168h)


In [23]:

# Test comparaison avec/sans filtrage
print(f"\n TEST COMPARAISON (échantillon 3 sources):")
test_sources = RSS_SOURCES[:3]

# Sans filtrage
collector_test = RSSCollector(sources=test_sources, days_back=7)
articles_filtered = collector_test.fetch_feeds(filter_by_date=True)
articles_all = collector_test.fetch_feeds(filter_by_date=False)

print(f"   Sans filtrage: {len(articles_all)} articles")
print(f"   Avec filtrage: {len(articles_filtered)} articles")
if len(articles_all) > 0:
    retention = len(articles_filtered) / len(articles_all) * 100
    print(f"   Taux rétention: {retention:.1f}%")
    print(f"   Articles éliminés: {len(articles_all) - len(articles_filtered)}")


 TEST COMPARAISON (échantillon 3 sources):

INFO:data.rss_collector: Collecte articles depuis: 2025-07-19 08:23
INFO:data.rss_collector: Lecture flux RSS: https://feeds.lemonde.fr/lemonde/actualites-en-continu/


INFO:data.rss_collector: 0 articles récupérés de https://feeds.lemonde.fr/lemonde/actualites-en-continu/
INFO:data.rss_collector: Lecture flux RSS: https://www.franceinter.fr/rss/a-la-une.xml
INFO:data.rss_collector: 0 articles récupérés de https://www.franceinter.fr/rss/a-la-une.xml
INFO:data.rss_collector: Lecture flux RSS: https://feeds.feedburner.com/lepoint/actualites
INFO:data.rss_collector: 0 articles récupérés de https://feeds.feedburner.com/lepoint/actualites
INFO:data.rss_collector:==================================================
INFO:data.rss_collector: STATISTIQUES DE COLLECTE RSS
INFO:data.rss_collector: Articles trouvés total    : 0
INFO:data.rss_collector: Articles filtrés (trop anciens): 0
INFO:data.rss_collector: Articles sans date        : 0
INFO:data.rss_collector: Sources en erreur         : 0
INFO:data.rss_collector: Articles finaux collectés : 0
INFO:data.rss_collector:==================================================
INFO:data.rss_collector: Lecture flux RSS: 

   Sans filtrage: 0 articles
   Avec filtrage: 0 articles


In [24]:

# RÉSUMÉ FINAL
print("\n" + "=" * 60)
print(" RÉSUMÉ DE LA COLLECTE")
print("=" * 60)
print(f" Sources RSS              : {len(RSS_SOURCES)}")
print(f" Articles trouvés          : {len(articles_rss)}")
print(f" Parsing réussi           : {success_count}")
print(f" Articles sauvés          : {saved_count}")
print(f" Total en base            : {len(all_articles)}")
print(f" Taux de succès           : {saved_count/len(articles_rss)*100:.1f}%")
print("=" * 60)




 RÉSUMÉ DE LA COLLECTE
 Sources RSS              : 26
 Articles trouvés          : 547
 Parsing réussi           : 432
 Articles sauvés          : 547
 Total en base            : 586
 Taux de succès           : 100.0%


In [25]:
if len(all_articles) >= 10:
    print(" COLLECTE RÉUSSIE ! Vous pouvez maintenant lancer l'enrichissement.")
else:
    print("  COLLECTE INSUFFISANTE. Vérifiez les sources RSS.")



 COLLECTE RÉUSSIE ! Vous pouvez maintenant lancer l'enrichissement.


In [26]:
# Nettoyage
session.close()
print(f" Collecte terminée à {datetime.now()}")

 Collecte terminée à 2025-07-26 08:23:37.355089
